# Creating a basic Neural Networks
In this notebook I using tensorflow, to create a DNN with subclassing API.

The data is sintetic, only to understand how the library works...

In [39]:
import os
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [2]:
def sintetic_data(shape:tuple):
    X = tf.random.uniform(shape=shape)
    y= tf.random.normal(shape=shape)
    return X,y 

X_train,y_train = sintetic_data(shape = (100,1))
X_val,y_val = sintetic_data(shape = (20,1))
X_test,y_test = sintetic_data(shape = (10,1))

In [3]:

class BasicModel(tf.keras.Model):
    def __init__(self,units, activation,num_hidden_states,**kwargs):
        super().__init__(**kwargs)
        self.num_hidden_states=num_hidden_states
        self.input_layer = tf.keras.Input(shape=(1,))
        self.dense1 = tf.keras.layers.Dense(units, activation=activation,input_shape = [1])
        self.dense2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
    
    def call(self, inputs):
        dense1 = self.dense1(inputs)
        dense2 = self.dense2(dense1)
        for i in range(self.num_hidden_states - 1):
            dense2 = self.dense2(dense2)
        output = self.main_output(dense2)
        return output
    
model = BasicModel(units=100, activation="relu",num_hidden_states=2)

loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss=loss_fn,
              optimizer=optimizer,
              metrics=["mae"])
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=100,verbose=False)

c:\Users\rafin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
model.evaluate(X_test,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.3556 - mae: 0.8486


[1.3556076288223267, 0.8486039042472839]

In [5]:
model.predict(tf.random.uniform(shape=(10,1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


array([[ 0.0651329 ],
       [-0.04995131],
       [ 0.20862377],
       [ 0.10885155],
       [ 0.24045259],
       [ 0.22787458],
       [ 0.13503286],
       [ 0.21613047],
       [ 0.11613606],
       [ 0.05841366]], dtype=float32)

In [9]:
def build_model(hp):  
  hp_units = hp.Int('units', min_value=4, max_value=128, step=4)  # Search for units between 4 and 128 in steps of 4
  hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'elu'])  # Search for these activation functions
  hp_num_hidden_states = hp.Int('num_hidden_states', min_value=2, max_value=5)  # Search for number of hidden layers (1 to 5)
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model = BasicModel(units=hp_units, activation=hp_activation, num_hidden_states=hp_num_hidden_states)
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='mse', metrics=['mae'])  # Compile with desired optimizer, loss, and metrics

  return model

In [44]:
tuner = kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=10,
                     directory='tuning',
                     project_name='basic_model')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

In [45]:
tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val),callbacks=[stop_early])

FailedPreconditionError: tuning\basic_model is not a directory

In [ ]:
best_models = tuner.get_best_models(num_models=2)  

best_model = best_models[0]
best_model.evaluate(X_test, y_test)